In [1]:
import numpy as np
import matplotlib.pyplot as plt

from distributions.distribution_by_sequence import DistributionBySequence
from distributions.distribution import Distribution
from distributions.sequence import Sequence
from algorithms.semi_bandit_exp3 import SemiBanditExp3
from algorithms.uniform_random import UniformRandom
from experiment_manager.experiment_manager import ExperimentManager

from distributions.contexts.binary_context import BinaryContext
from distributions.thetas.single_hole import SingleHole

In [2]:
exp_manager = ExperimentManager()
algos = [SemiBanditExp3(), UniformRandom()]

length = 2000
d = 2
K = 2
actionset = [np.ones(K, dtype=bool)]
for i in range(K):
    buffer = np.zeros(K, dtype=bool)
    buffer[i] = 1
    actionset.append(buffer)
actionset = np.array(actionset)

dist = Distribution(BinaryContext(d), SingleHole(K, d, np.array([0.7, 0.3])), actionset)

In [3]:
exp_manager.run(1, [2000], algos, [dist])

array([[234.],
       [543.]])